<a href="https://colab.research.google.com/github/Tinynja/Sarsa-phi-EB/blob/main/ALE_Framework_Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

if 'google.colab' in sys.modules:
    !rm -rf *
    !git clone https://github.com/Tinynja/Sarsa-phi-EB
    !mv Sarsa-phi-EB/* .
    !rm -rf Sarsa-phi-EB
    # DON'T install packages defined in Pipfile_colab_remove
    !sed -ri "/$(tr '\n' '|' < Pipfile_colab_remove)/d" Pipfile
else:
    print('Skipping GitHub cloning since not running in Colab.')

Cloning into 'Sarsa-phi-EB'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 184 (delta 35), reused 148 (delta 17), pack-reused 0
Receiving objects: 100% (184/184), 540.49 KiB | 1.54 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [2]:
# Install required dependencies
if 'google.colab' in sys.modules:
    # Colab doesn't support pipenv, hence we convert Pipfile into requirements.txt
    !pip install pipenv 1> /dev/null
    !pipenv lock -r > requirements.txt
    !pip install -r requirements.txt 1> /dev/null
else:
    !pipenv lock 1> /dev/null
    !pipenv install --deploy 1> /dev/null

Pipfile.lock not found, creating...
Locking [dev-packages] dependencies...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! 
Updated Pipfile.lock (44ce9b)!


In [3]:
# Import all supported ROMs into ALE
!ale-import-roms ROMS

[SUPPORTED]                    qbert        ROMS/Q. Bert (1983).bin
[SUPPORTED]                   casino ROMS/Casino - Poker Plus (Paddle) (1979).bin
[SUPPORTED]             demon_attack ROMS/Demon Attack (Death from Above) (1982).bin
[SUPPORTED]                  bowling        ROMS/Bowling (1979).bin
[SUPPORTED]                 seaquest       ROMS/Seaquest (1983).bin
[SUPPORTED]          chopper_command ROMS/Chopper Command (1982).bin
[SUPPORTED]               time_pilot     ROMS/Time Pilot (1983).bin
[SUPPORTED]                 trondead ROMS/TRON - Deadly Discs (TRON Joystick) (1983).bin
[SUPPORTED]                jamesbond ROMS/James Bond 007 (James Bond Agent 007) (1984).bin
[SUPPORTED]                   pacman        ROMS/Pac-Man (1982).bin
[SUPPORTED]                   kaboom ROMS/Kaboom! (Paddle) (1981).bin
[SUPPORTED]                  asterix ROMS/Asterix (AKA Taz) (1983).bin
[SUPPORTED]                    qbert         ROMS/Q-bert (1987).bin
[SUPPORTED]                  pitfal

In [4]:
# Utilities for visualizing a simulation
import base64
from IPython import display as ipythondisplay

def show_video(filepath):
    """Show a .mp4 video in html format
    
    Example usage: show_video('record.mp4')
    """
    mp4 = Path(filepath)
    video_b64 = base64.b64encode(mp4.read_bytes())
    html = '''<video alt="{}" autoplay loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{}" type="video/mp4" />
              </video>'''.format(mp4, video_b64.decode('ascii'))
    ipythondisplay.display(ipythondisplay.HTML(data=html))

def png_to_mp4(file_pattern, output):
    """Convert a set of png files into a mp4 video

    Arguments:
    file_pattern -- pattern matching the name of the input png
    output -- output path of the mp4
    
    Example usage: png_to_mp4('record/step_%03d.png', 'record.mp4')
    """
    input_full = Path(file_pattern).resolve()
    input_directory = input_full.parent
    input_pattern = input_full.name
    output_full = Path(output).resolve()
    !cd $input_directory; ffmpeg -hide_banner -loglevel error -r 60 -i $input_pattern -vcodec libx264 -crf 25 -pix_fmt yuv420p -y $output_full

In [5]:
# Built-in libraries
import sys
import random
from pathlib import Path

# Pipy libraries
from ale_py import ALEInterface, SDL_SUPPORT
from ale_py.roms import Breakout

/usr/local/lib/python3.7/dist-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [20]:
# Setup
ale = ALEInterface()
ale.setInt("random_seed", 0)

if SDL_SUPPORT and 'google.colab' not in sys.modules:
    ale.setBool("sound", True)
    ale.setBool("display_screen", True)

ale.loadROM(Breakout)
legal_actions = ale.getLegalActionSet()
print(legal_actions)

# Export
!rm record/*
Path('record').mkdir(exist_ok=True)

# Play 10 episodes
for episode in range(1):
    total_reward = 0
    j = 0
    while not ale.game_over():
        ale.saveScreenPNG(f'record/step_{j:04d}.png')
        #action = random.choice(legal_actions) #random policy
        if j%3 == 0:
          action = legal_actions[4]
        elif j%3 == 1:
          action = legal_actions[3]
        else:
          action = legal_actions[1]
        # Apply an action and get the resulting reward
        reward = ale.act(action)
        total_reward += reward
        j += 1
    print("Episode %d ended with score: %d" % (episode, total_reward))
    ale.reset_game()

[<Action.NOOP: 0>, <Action.FIRE: 1>, <Action.UP: 2>, <Action.RIGHT: 3>, <Action.LEFT: 4>, <Action.DOWN: 5>, <Action.UPRIGHT: 6>, <Action.UPLEFT: 7>, <Action.DOWNRIGHT: 8>, <Action.DOWNLEFT: 9>, <Action.UPFIRE: 10>, <Action.RIGHTFIRE: 11>, <Action.LEFTFIRE: 12>, <Action.DOWNFIRE: 13>, <Action.UPRIGHTFIRE: 14>, <Action.UPLEFTFIRE: 15>, <Action.DOWNRIGHTFIRE: 16>, <Action.DOWNLEFTFIRE: 17>]
Episode 0 ended with score: 2


In [21]:
png_to_mp4('record/step_%04d.png', 'record.mp4')
show_video('record.mp4')